# Output RDMBS Alchemy

This component pushes data to any RDMBS supported by SQLAlchemy as CSV on a given table. Parameters like host, database, user, password and table name need to be set.

Currently only append mode is supported via the execute_batch helper. Optionally, you can flush (truncate) the table before.

The current implementation loads all data into main memory (via pandas) first. Better implementations can be found below, PRs welcome!

https://hakibenita.com/fast-load-data-python-postgresql

In [1]:
!pip install sqlalchemy==1.4.23 pandas==1.3.1 psycopg2-binary==2.9.1

  Using cached https://files.pythonhosted.org/packages/38/86/209b810ec1d6a041eb60985ff3a3c671633bcc5665b93ed8467d7bd11987/SQLAlchemy-1.4.23-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl


  Using cached https://files.pythonhosted.org/packages/1d/99/80783b636b98a66f9e9ae39d6691b21efc1f293f9ed56c2ef8d17753dccd/pandas-1.3.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


  Found existing installation: SQLAlchemy 1.4.29


    Uninstalling SQLAlchemy-1.4.29:
      Successfully uninstalled SQLAlchemy-1.4.29


  Found existing installation: pandas 1.3.5


    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import logging
import os
import pandas as pd
import re
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import sys

In [3]:
# data to load (expects CSV file with header)
data_csv = os.environ.get('data_csv', 'data.csv')

# type of database server (sqlalchemy dialect), e.g. postgresql
db_type = os.environ.get('db_type', 'postgresql')

# hostname of database server
host = os.environ.get('host')

# database name
database = os.environ.get('database')

# db user
user = os.environ.get('user')

# db password
password = os.environ.get('password')

# db port
port = int(os.environ.get('port', 5432))

# schema name
schema = os.environ.get('schema')

# table name
table = os.environ.get('table')

# drop table before insert
drop = bool(os.environ.get('drop', True))

# delimiter
delimiter = os.environ.get('delimiter', ',')

# character encoding
encoding = os.environ.get('encoding', 'cp1252')

# temporal data storage for local execution
data_dir = os.environ.get('data_dir', '../../data/')

In [4]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

drop = bool(drop)
port = int(port)

In [5]:
engine = create_engine(f'{db_type}://{user}:{password}@{host}:{port}/{database}')

In [6]:
if drop:
    with engine.connect() as con:
        con.execution_options(autocommit=True).execute(f'DROP TABLE IF EXISTS {schema}.{table};')

In [7]:
Session = sessionmaker(bind=engine)

with Session() as session:
    df = pd.read_csv(data_dir + data_csv, encoding=encoding, delimiter=delimiter)
    df.columns= df.columns.str.lower()
    df.columns = df.columns.str.replace('[ä,ö,ü]', '[ae,oe,ue]')
    df.to_sql(table, con=engine, if_exists='append', index=False)

/tmp/ipykernel_821604/1475509544.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[ä,ö,ü]', '[ae,oe,ue]')


In [8]:
#import pandas as pd
#df = pd.read_csv('../../data/12613-0004_flat.csv', encoding='cp1252', delimiter=';')
#df.columns= df.columns.str.lower()
#df.columns = df.columns.str.replace('[ä,ö,ü]', '[ae,oe,ue]')